# Coffee habits around the world

In present notebooks we would study the dataset `worldwide_coffee_habits` from [Waqar Ali](https://www.kaggle.com/datasets/waqi786/worldwide-coffee-habits-dataset/data). Here, we will explore it to find interesting facts about coffee trends

In [14]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

## Cleaning and transforming

First, we must review data and check if it's ready to be analyzed

In [15]:
df = pd.read_csv("data/worldwide_coffee_habits.csv")
df.shape

(10000, 6)

In [16]:
df.sample(5)

,Country,Year,Coffee Consumption (kg per capita per year),Average Coffee Price (USD per kg),Type of Coffee Consumed,Population (millions)
7643,Country_3,2017,2.293975,7.380966,Espresso,17.538839
824,Country_26,2010,9.402638,6.499693,Latte,110.268323
3686,Country_21,2005,5.107496,10.585250,Latte,62.307352
9907,Country_8,2017,3.473508,7.368068,Mocha,148.578701
4567,Country_34,2012,5.584729,11.752544,Mocha,112.657718


The country can be improved using only the code, extract such data from **Country** column

In [17]:
df["Country"] = df.Country.str.extract( r'_(\d+)', expand=False)

In [18]:
# Change columns names and adjust data type of Country
df.columns = ['Country', "Year", 'Consumption', 'Price', 'Type', 'Population']
df.Country = df.Country.astype('int64')

Check some data about categorical and numerical variables

In [19]:
print("Tipos de cafe: ", df.Type.unique())
print("Numero de paises: ", df.Country.unique().shape[0])

Tipos de cafe:  ['Americano' 'Mocha' 'Latte' 'Espresso' 'Cappuccino']
Numero de paises:  50


In [20]:
df.describe()

,Country,Year,Consumption,Price,Population
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,25.620400,2011.666900,6.061865,9.461891,75.167120
std,14.343489,6.911695,2.313427,3.151403,43.023176
min,1.000000,2000.000000,2.000385,4.000742,1.002494
25%,13.000000,2006.000000,4.070743,6.728261,37.465847
50%,26.000000,2012.000000,6.094491,9.458371,75.021943
75%,38.000000,2018.000000,8.061127,12.136285,112.595868
max,50.000000,2023.000000,9.999399,14.997053,149.995850


This way, we see that have balanced data around the years.

Now, the data doesn't have any null values or repited, so let's explore some data about a country

In [21]:
df[df.Country == 15].sort_values('Year')[:10]

,Country,Year,Consumption,Price,Type,Population
934,15,2000,2.514999,6.284871,Espresso,91.724444
1242,15,2000,6.627786,6.604495,Mocha,96.916388
5030,15,2000,4.923407,10.595365,Cappuccino,43.454157
3401,15,2000,5.461281,11.543387,Latte,79.441216
5388,15,2000,9.055128,5.669860,Mocha,136.560499
8160,15,2000,3.980497,9.915708,Espresso,130.282176
8391,15,2000,9.494177,5.564295,Cappuccino,126.455857
7244,15,2000,6.409317,9.049182,Espresso,48.203251
3365,15,2001,7.390923,10.649485,Cappuccino,122.257420
5441,15,2001,6.089285,9.597600,Americano,44.524898


Here, we must note something strange. For a country in the same year, we have rows that do not repeat but for the same country report, in the same year and a type of coffe, we have different values on Consumption and most important, Population.

This behavior is not explain it by the source, so we will solve this problem calculating the means

In [9]:
# Create new dataframe
df2 = pd.DataFrame(columns=df.columns)
for year in df.Year.unique():
    for country in df.Country.unique():
        for tpe in df.Type.unique():
            # For each category, get the mean and update the new dataframe
            consp, price, pop = df[['Consumption', 'Price', 'Population']][(df.Country == country) & (df.Year == year) & (df.Type == tpe)].mean()
            df2.loc[len(df2), :] = [country, year, consp, price, tpe, pop]

In [10]:
# Check dataframe info again
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6000 entries, 0 to 5999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Country      6000 non-null   object
 1   Year         6000 non-null   object
 2   Consumption  4860 non-null   object
 3   Price        4860 non-null   object
 4   Type         6000 non-null   object
 5   Population   4860 non-null   object
dtypes: object(6)
memory usage: 457.2+ KB


In [11]:
# Let's check those null values
df2[df2.Price.isna()][:10]

,Country,Year,Consumption,Price,Type,Population
1,39,2023,NaN,NaN,Mocha,NaN
11,15,2023,NaN,NaN,Mocha,NaN
16,43,2023,NaN,NaN,Mocha,NaN
18,43,2023,NaN,NaN,Espresso,NaN
19,43,2023,NaN,NaN,Cappuccino,NaN
20,8,2023,NaN,NaN,Americano,NaN
25,21,2023,NaN,NaN,Americano,NaN
27,21,2023,NaN,NaN,Latte,NaN
29,21,2023,NaN,NaN,Cappuccino,NaN
42,11,2023,NaN,NaN,Latte,NaN


In [12]:
# Because they aren't meaningfull, discard them
df2.dropna(inplace=True)

In [13]:
# Now, we have a better dataframe to work
df2.sample(10)

,Country,Year,Consumption,Price,Type,Population
5127,16,2009,9.128179,13.849297,Latte,36.645143
5518,43,2003,5.045448,8.402199,Espresso,56.885115
632,47,2020,6.965339,11.243186,Latte,55.217284
2781,19,2007,6.867263,7.115016,Mocha,74.353629
2477,1,2021,4.680332,7.826503,Latte,41.851389
5863,27,2014,5.63635,8.221621,Espresso,68.586875
1475,1,2004,6.339221,9.282736,Americano,79.085055
2371,28,2021,3.900621,11.29013,Mocha,35.374457
126,16,2023,4.547944,7.969579,Mocha,108.206705
1508,29,2022,5.667311,5.861533,Espresso,140.953546


## Analysis

In [22]:
# Making some adjustments
df2 = df2.sort_values("Year")
df2['Total Consumption'] = df2.Consumption * df2.Population

In [23]:
# Coffee distribution of year by top 5 countries
year = 2023
tmp = df2[df2.Year == year]
top5 = tmp[['Total Consumption', 'Country']].groupby('Country').sum().sort_values('Total Consumption', ascending=False).index[:5]
fig = px.pie(tmp[tmp.Country.isin(top5)], 'Type', 'Total Consumption')
fig.update_layout(title=f"Coffee Type distribution by consumption in top 5 countries in {year}")
fig.show()

In [24]:
# Most dominant coffee globally by market value and year
## Filter data and sum it
mkv = df2[['Year', 'Price', 'Total Consumption', 'Type']].groupby(["Year",'Type']).sum()
# Get total market revenue
mkv['Capitalization'] = mkv.Price * mkv['Total Consumption']
# Drop data and Unstack it
mkv = mkv.drop(['Price', 'Total Consumption'], axis=1).unstack()['Capitalization']
# Resamble df to have most consumed coffee type by year
mcc = pd.DataFrame(columns = ['Type', 'Capitalization'], index=mkv.index)
for i, m in enumerate(mkv.max(axis=1)):
    mcc.loc[mkv.index[i], ["Type", 'Capitalization']] = [mkv.columns[mkv.iloc[i] == m].values[0], m]
mcc.reset_index(inplace=True)
# Plot trends
fig = go.Figure()
for t in mcc.Type.unique():
    fig.add_trace(go.Bar(x=mcc.Year[mcc.Type == t], y=mcc.Capitalization[mcc.Type == t], name=t))

fig.update_layout(title="Most Consumed Coffee Globally by Capitalization")
fig.show()

Here we can see how the *Cappuccino* is the most consumed coffee in last three years. Also, we see a downside trend in *Capitalization* since 2020. With a detailed data across months, it could be possible to see if the Covid-19 pandemic is the main reason of it.

In [25]:
# Most expensive coffee globally
## Filter data and get the mean
mkv = df2[['Year', 'Price', 'Type']].groupby(["Year",'Type']).mean()
mkv = mkv.unstack()['Price']
# Resamble df to have most expensive coffee type by year
mec = pd.DataFrame(columns = ['Type', 'Price'], index=mkv.index)
for i, m in enumerate(mkv.max(axis=1)):
    mec.loc[mkv.index[i], ["Type", 'Price']] = [mkv.columns[mkv.iloc[i] == m].values[0], m]
mec.reset_index(inplace=True)
mec.Price = mec.Price.astype('float64')
# Plot trends
fig = go.Figure()
for t in mec.Type.unique():
    fig.add_trace(go.Bar(x=mec.Year[mec.Type == t], y = mec.Price[mec.Type == t], name=t))

fig.update_layout(title="Most Expensive Coffee Globally acrross the years")
fig.update_yaxes(range=(9.6, 10.4))
fig.show()

In [26]:
# General price of coffee across the years
mpy = df2[['Year', 'Price']].groupby('Year').mean()
fig = go.Figure(go.Scatter(x = mpy.index, y = mpy.Price, mode='lines'))
fig.update_layout(title='Mean price of Coffee across the years')
fig.show()

With this plot we can a recent downside trend in coffee prices

In [27]:
# Price across the years for more populated and least populated countries
prices_top_botton = pd.DataFrame(columns=['Year', 'Top', 'Bottom'])
for year in df2.Year.unique():
    cbp = df2[['Country', 'Population']][df2.Year == year].groupby('Country').sum().sort_values('Population')
    top10 = df2.Price[(df2.Country.isin(cbp.index[-10:])) & (df2.Year == year)].mean()
    bottom10 = df2.Price[(df2.Country.isin(cbp.index[:10])) & (df2.Year == year)].mean()
    prices_top_botton.loc[len(prices_top_botton), :] = [year, top10, bottom10]

fig = go.Figure(go.Scatter(x = prices_top_botton.Year, y = prices_top_botton.Top, 
                           name='Mean Price Top 10', mode='lines'))
fig.add_trace(go.Scatter(x = prices_top_botton.Year, y = prices_top_botton.Bottom, 
                         name='Mean Price Bottom 10', mode='lines'))
fig.update_layout(title='Price across the years of top and bottom countries by population')
fig.show()

Here we can that the prices are very similar but their variations are almost inversed. Such pattern may indicate the global performance of coffee and how it's stabilize on time

In [28]:
# Consumption across the years for more populated and least populated countries
cons_top_botton = pd.DataFrame(columns=['Year', 'Top', 'Bottom'])
for year in df2.Year.unique():
    cbp = df2[['Country', 'Population']][df2.Year == year].groupby('Country').sum().sort_values('Population')
    top10 = df2.Consumption[(df2.Country.isin(cbp.index[-10:])) & (df2.Year == year)].mean()
    bottom10 = df2.Consumption[(df2.Country.isin(cbp.index[:10])) & (df2.Year == year)].mean()
    cons_top_botton.loc[len(cons_top_botton), :] = [year, top10, bottom10]

fig = go.Figure(go.Scatter(x = cons_top_botton.Year, y = cons_top_botton.Top, 
                           name='Mean Consumption Top 10', mode='lines'))
fig.add_trace(go.Scatter(x = cons_top_botton.Year, y = cons_top_botton.Bottom, 
                         name='Mean Consumption Bottom 10', mode='lines'))
fig.update_layout(title='Consumption across the years of top and bottom countries by population')
fig.show()

From the plot, we can see how the mean consumption of coffee in least populated countries is lower than the more populated. Also, the consequences from Covid-19 pandemic represent a big collapse in consumption of coffee in the least populated countries, perhaps related to limited access to inputs due to their limited economic resources.

In [29]:
# Number of times type of coffee is dominant by country across the years
# Create stats holder
stats = [{'Americano': 0, 'Mocha': 0, 'Espresso': 0, 'Latte': 0, 'Cappuccino':0} for i in range(50)]
for year in df2.Year.unique():
    # For each year, get the max value of consumption by country
    tmp = df2[['Country', 'Total Consumption']][df2.Year == year].groupby(['Country']).max()
    for i, r in tmp.iterrows():
        # Get the type of coffee with max consumption and add it to stats holder
        t = df2['Type'][(df2.Year == year) & (df2.Country == i) & (df2['Total Consumption'] == r.iloc[0])].values[0]
        stats[i-1][t] += 1

# Show data
for i in range(50):
    coffee = max(stats[i], key=lambda x: stats[i][x])
    number = round(max(stats[i].values())*100 / sum(stats[i].values()))
    print(f'For country {i+1} the most consumed coffe across the years is: {coffee} with {number}% of years\n')

For country 1 the most consumed coffe across the years is: Americano with 29% of years

For country 2 the most consumed coffe across the years is: Espresso with 33% of years

For country 3 the most consumed coffe across the years is: Americano with 29% of years

For country 4 the most consumed coffe across the years is: Cappuccino with 25% of years

For country 5 the most consumed coffe across the years is: Americano with 38% of years

For country 6 the most consumed coffe across the years is: Latte with 29% of years

For country 7 the most consumed coffe across the years is: Espresso with 26% of years

For country 8 the most consumed coffe across the years is: Latte with 25% of years

For country 9 the most consumed coffe across the years is: Cappuccino with 42% of years

For country 10 the most consumed coffe across the years is: Espresso with 29% of years

For country 11 the most consumed coffe across the years is: Espresso with 33% of years

For country 12 the most consumed coffe a

In [30]:
# Consumption relation with coffee type
## Calculate market total value (millions dollars by year)
ct = 'Latte'
df2['market_value'] = df2.Price * df2.Consumption * df2.Population
# Get pivot table with stats and plot
tmp = df2.pivot_table(['market_value', 'Population'], 'Country', 'Type').reset_index()
fig = go.Figure(go.Scatter(x=tmp.index, y = tmp['market_value'][ct], mode='markers',
                           marker = dict(size=14, color=tmp['Population'][ct], showscale=True)))
fig.update_layout(title=f'Market value (Million dollar by year) by country and population for {ct}')
fig.show()

Exploring data with this last tool across the coffee types we can see how the *market value* is not related to the country, type or population. This fact reflects that all variables in the dataset have poor correlation, this means we can not perform ML methods to get future outcomes.

In [31]:
# Save datas
df2.to_csv('data/Coffee_ready_data.csv', index=False)